In [12]:
def post_process(file):
    with open(file, "r", encoding="utf-8") as f:
        content = f.readlines()

    list_start = []
    for idx, line in enumerate(content):
        if "|Col1|VIETTEL AI RACE" in line:
            list_start.append(idx)
    to_remove = set()
    for s in list_start:
        for j in (s, s + 1, s + 2):
            if 0 <= j < len(content):
                to_remove.add(j)

    content = [line for i, line in enumerate(content) if i not in to_remove]
    new_text = "\n".join(content)
    return new_text

In [ ]:
import os
list_orin_file = "output/var_test_pymu"
list_post_file = list_orin_file+"_post"
os.makedirs(list_post_file, exist_ok=True)
for file in os.listdir(list_orin_file):
    if file.endswith(".md"):
        orin_path = os.path.join(list_orin_file, file)
        post_path = os.path.join(list_post_file, file)
        md = post_process(orin_path)
        with open(post_path, "w", encoding="utf-8") as f:
            f.write(md)
